In [1]:
import pandas as pd

In [2]:
test_df = pd.read_csv("/kaggle/input/recipe-test-data/test_data.csv", index_col = "Unnamed: 0")
test_df.head(2)

,Instruction,Response,TF_IDF_GeneratedRecipe
0,"Tags: ['60-minutes-or-less', 'time-to-make', '...",Name: french toast with a crunchy topping Minu...,Name: apple a day milk shake Minutes: 0 Ingre...
1,"Tags: ['weeknight', 'time-to-make', 'course', ...",Name: almost authentic cincinnati chili Minute...,Name: forgotten minestrone Minutes: 495 Ingre...


## Fine-Tuned BART

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="SaiSamyuktaPalle/RecipeGeneration-BART")

2024-04-26 01:56:02.360708: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 01:56:02.360828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 01:56:02.517839: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("SaiSamyuktaPalle/RecipeGeneration-BART")
model = AutoModelForSeq2SeqLM.from_pretrained("SaiSamyuktaPalle/RecipeGeneration-BART")

In [5]:
def generate_recipe(tokenizer, model, prompt, max_input_length=64, max_output_length=512, num_beams=5, no_repeat_ngram_size=2):
    """
    Generate a recipe based on the given prompt.

    Args:
        tokenizer: The tokenizer object.
        model: The recipe generation model.
        prompt (str): The input prompt for generating the recipe.
        max_input_length (int): Maximum length of input sequence. Default is 64.
        max_output_length (int): Maximum length of output sequence. Default is 512.
        num_beams (int): Number of beams for beam search. Default is 5.
        no_repeat_ngram_size (int): Size of n-grams for preventing repetition. Default is 2.

    Returns:
        str: The generated recipe text.
    """
    # Encode the text using the tokenizer
    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=max_input_length, truncation=True)
    
    # Generate outputs
    output_sequences = model.generate(
        inputs, 
        max_length=max_output_length, 
        num_beams=num_beams, 
        no_repeat_ngram_size=no_repeat_ngram_size, 
        early_stopping=True
    )
    
    # Decode the output sequences to get the generated text
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    
    return generated_text

In [6]:
# Example input text
prompt = "Tags: ['30-minutes-or-less', 'vegetarian'] Ingredients: 'onion', 'mushroom pieces'"
generated_text = generate_recipe(tokenizer, model, prompt, max_input_length=64, max_output_length=512, num_beams=5, no_repeat_ngram_size=2)
print(generated_text)

: Name: creamy mushroom and mushroom saute Minutes: 30 Ingredients: ['onion','mushroom pieces', "confectioners' sugar",'vegetable oil'] Steps: ["saute' onion and mushrooms in vegetable oil in a large skillet until onion is tender, but not browned", "add confectioner's sugar and continue cooking until the sugar is dissolved and the mushrooms are cooked through, about 10 minutes, stirring occasionally, so it doesn't stick to the bottom of the pan"]


In [7]:
from tqdm import tqdm

generated_texts = []
for prompt in tqdm(test_df['Instruction'].to_list(), desc="Generating Recipes"):
    generated_text = generate_recipe(tokenizer, model, prompt, max_input_length=64, max_output_length=512, num_beams=5, no_repeat_ngram_size=2)
    generated_texts.append(generated_text)

Generating Recipes: 100%|██████████| 100/100 [18:59<00:00, 11.39s/it]


In [8]:
test_df['FineTuned_BART_GeneratedRecipe'] = generated_texts

## Pre-trained BART

In [9]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the pre-trained BART tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
# Load the pre-trained BART model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [10]:
generated_texts_pre = []
for prompt in tqdm(test_df['Instruction'].to_list(), desc="Generating Recipes"):
    generated_text = generate_recipe(tokenizer, model, prompt, max_input_length=64, max_output_length=512, num_beams=5, no_repeat_ngram_size=2)
    generated_texts_pre.append(generated_text)

Generating Recipes: 100%|██████████| 100/100 [07:55<00:00,  4.75s/it]


In [11]:
test_df['PreTrained_BART_GeneratedRecipe'] = generated_texts_pre

In [12]:
test_df.head()

,Instruction,Response,TF_IDF_GeneratedRecipe,FineTuned_BART_GeneratedRecipe,PreTrained_BART_GeneratedRecipe
0,"Tags: ['60-minutes-or-less', 'time-to-make', '...",Name: french toast with a crunchy topping Minu...,Name: apple a day milk shake Minutes: 0 Ingre...,: Name: best waffles ever Minutes: 40 Ingredie...,"Tags: ['60-minutes-or-less', 'time-to-make', '..."
1,"Tags: ['weeknight', 'time-to-make', 'course', ...",Name: almost authentic cincinnati chili Minute...,Name: forgotten minestrone Minutes: 495 Ingre...,: Name: beef stroganoff Minutes: 255 Ingredien...,"Tags: ['weeknight', 'time-to-make', '/course',..."
2,"Tags: ['15-minutes-or-less', 'time-to-make', '...",Name: amazing watermelon greek salad with feta...,Name: apple a day milk shake Minutes: 0 Ingre...,: Name: german salad Minutes: 15 Ingredients: ...,"Tags: ['15-minutes-or-less', 'time-to-make', ‘..."
3,"Tags: ['60-minutes-or-less', 'time-to-make', '...",Name: chicken tetrazzini Minutes: 55 Ingredien...,Name: crispy crunchy chicken Minutes: 35 Ingr...,: Name: chicken and noodles Minutes: 45 Ingred...,"Tags: ['60-minutes-or-less', 'time-to-make','c..."
4,"Tags: ['weeknight', 'time-to-make', 'course', ...",Name: chicken with white wine garlic Minutes...,Name: forgotten minestrone Minutes: 495 Ingre...,: Name: chicken souvlaki Minutes: 255 Ingredie...,"Tags: ['weeknight', 'time-to-make"", 'course', ..."


## BLEU Score

In [13]:
from nltk.translate.bleu_score import sentence_bleu

In [14]:
def avg_bleu_score(original_texts, generated_texts):
    # Initialize a list to store BLEU scores
    bleu_scores = []

    # Iterate over each pair of original and generated texts
    for original_text, generated_text in zip(original_texts, generated_texts):
        # Tokenize the original and generated texts
        original_tokens = original_text.split()  # You can use NLTK's word_tokenize for more advanced tokenization
        generated_tokens = generated_text.split()  # Similarly, tokenize the generated text

        # Calculate BLEU score
        bleu_score = sentence_bleu([original_tokens], generated_tokens)
        bleu_scores.append(bleu_score)

    # Calculate the average BLEU score
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu_score


In [15]:
original_texts = test_df.Instruction.to_list()  # List of original texts
generated_texts1 = test_df.FineTuned_BART_GeneratedRecipe.to_list() # List of generated texts
generated_texts2 = test_df.PreTrained_BART_GeneratedRecipe.to_list() # List of generated texts
generated_texts3 = test_df.TF_IDF_GeneratedRecipe.to_list() # List of generated texts

print(f"Average BLEU Score for FineTuned BART: {avg_bleu_score(original_texts, generated_texts1)}")
print(f"Average BLEU Score for PreTrained BART: {avg_bleu_score(original_texts, generated_texts2)}")
print(f"Average BLEU Score for TF_IDF: {avg_bleu_score(original_texts, generated_texts3)}")

Average BLEU Score for FineTuned BART: 0.3317397323707971
Average BLEU Score for PreTrained BART: 0.03736246884315665
Average BLEU Score for TF_IDF: 0.3346247108507307


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


## Qualitative Analysis

Downloading the top 20 rows of test_df with the results, and doing Human Evaluation on them.

In [16]:
test_df.iloc[:20,:].to_csv('inference.csv')